<a href="https://colab.research.google.com/github/mhtattersall/mhtattersall/blob/main/TechnicalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python code and comments for MACD, Bollinger Bands, Average True Range, Relative Strength Index, Average Directional Index and Renko.  Based on lectures from Mayak Rassu (Udemy) and approximating to tradingview.com charts.

In [12]:
# Import necesary libraries
import yfinance as yf
import pandas as pd
import numpy as np

In [14]:
# Download historical data for required stocks
tickers = ["MSFT","AAPL","GOOG"]
ohlcv_data = {} #empty dictionary
# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m') #15 or 5 minute intervals over 1 month, may need to adjust 15m to match other published metrics
    temp.dropna(how="any",inplace=True) #drop rows wioth Nan values
    ohlcv_data[ticker] = temp #populate the dictionary with a df for each stock with its ticker as the key
print(ohlcv_data["MSFT"]) #print df for MSFT

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-09 09:30:00-04:00  324.750000  325.199799  323.589996  324.149994   
2023-10-09 09:35:00-04:00  324.174988  324.540009  323.179993  324.174988   
2023-10-09 09:40:00-04:00  324.183807  325.714996  323.850006  325.570007   
2023-10-09 09:45:00-04:00  325.579987  326.859985  325.470001  326.839996   
2023-10-09 09:50:00-04:00  326.829987  327.179993  326.190002  326.850006   
...                               ...         ...         ...         ...   
2023-11-08 09:30:00-05:00  361.679993  363.869995  361.404999  363.869995   
2023-11-08 09:35:00-05:00  363.850006  363.869995  362.239990  362.804993   
2023-11-08 09:40:00-05:

In [15]:
def MACD(DF, a=12 ,b=26, c=9):
    """function to calculate MACD (with exponential moving average)
       typical values a(fast moving average) = 12;
                      b(slow moving average) =26;
                      c(signal line ma window) =9"""
    df = DF.copy() #copy the df with the prices for the stock
    df["ma_fast"] = df["Adj Close"].ewm(span=a, min_periods=a).mean() #new column which calculates the fast moving average on the adjusted close price for the stock as a column
    df["ma_slow"] = df["Adj Close"].ewm(span=b, min_periods=b).mean() #new column which calculates the slow moving average for the stock
    df["macd"] = df["ma_fast"] - df["ma_slow"] #new column which calculates fast - slow averages for the stock
    df["signal"] = df["macd"].ewm(span=c, min_periods=c).mean() #new column, which calculates signal moving average for the stock using the macd
    return df.loc[:,["macd","signal"]] #return the MACD and signal columns only for each stock

for ticker in ohlcv_data:
    ohlcv_data[ticker][["MACD","SIGNAL"]] = MACD(ohlcv_data[ticker], a=12 ,b=26, c=9) #apply the function to each stock

In [ ]:
print(ohlcv_data["MSFT"]) #check these against other published values

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-04 09:30:00-04:00  314.029999  316.600006  314.000000  316.528503   
2023-10-04 09:35:00-04:00  316.559998  317.456207  316.350006  317.130005   
2023-10-04 09:40:00-04:00  317.149811  317.269989  316.625000  316.869995   
2023-10-04 09:45:00-04:00  316.880005  317.880005  316.730011  317.364990   
2023-10-04 09:50:00-04:00  317.339996  317.525909  316.070007  316.575012   
...                               ...         ...         ...         ...   
2023-11-03 15:35:00-04:00  353.720001  353.880005  353.579987  353.839996   
2023-11-03 15:40:00-04:00  353.850006  353.869995  353.510010  353.540009   
2023-11-03 15:45:00-04:00  353.539886  353.539886  352.929993  353.029999   
2023-11-03 15:50:00-04:00  353.079987  353.269989  352.660004  352.815002   
2023-11-03 15:55:00-04:00  352.825012  353.220001  352.609985  352.809998   

In [16]:
def Boll_Band(DF, n=14): #default value for window of rolling mean is 14 periods
    "function to calculate Bollinger Band"
    df = DF.copy() #ncopy the df with the prices for the stock
    df["MB"] = df["Adj Close"].rolling(n).mean() #new column which is the 14 day simple moving average
    df["UB"] = df["MB"] + 2*df["Adj Close"].rolling(n).std(ddof=0) #new columns which is the 14 day simple moving average + 2xSD on closing prices #ddof is degrees of freedom, which is set to 0 as it is the SD of a population
    df["LB"] = df["MB"] - 2*df["Adj Close"].rolling(n).std(ddof=0) #new column which is the 14 day simple movng average - 2xSD on closing prices
    df["BB_Width"] = df["UB"] - df["LB"] #new column which is upper band value minus the lower band
    return df[["MB","UB","LB","BB_Width"]] #return these 4 columns

for ticker in ohlcv_data:
    ohlcv_data[ticker][["MB","UB","LB","BB_Width"]] = Boll_Band(ohlcv_data[ticker]) #apply the function to each stock and for each column

In [17]:
print(ohlcv_data["MSFT"])

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-09 09:30:00-04:00  324.750000  325.199799  323.589996  324.149994   
2023-10-09 09:35:00-04:00  324.174988  324.540009  323.179993  324.174988   
2023-10-09 09:40:00-04:00  324.183807  325.714996  323.850006  325.570007   
2023-10-09 09:45:00-04:00  325.579987  326.859985  325.470001  326.839996   
2023-10-09 09:50:00-04:00  326.829987  327.179993  326.190002  326.850006   
...                               ...         ...         ...         ...   
2023-11-08 09:30:00-05:00  361.679993  363.869995  361.404999  363.869995   
2023-11-08 09:35:00-05:00  363.850006  363.869995  362.239990  362.804993   
2023-11-08 09:40:00-05:00  362.804993  362.869995  362.040009  362.265015   
2023-11-08 09:45:00-05:00  362.339996  363.050415  362.130005  362.755005   
2023-11-08 09:50:00-05:00  362.877899  362.877899  362.877899  362.877899   

In [18]:
def RSI(DF, n=14): #default value for window of rolling mean is 14 periods
    "function to calculate RSI"
    df = DF.copy() #copy the df with the prices for the stock
    df["change"] = df["Adj Close"] - df["Adj Close"].shift(1) #new cocalculate the change in price vs the previous period
    df["gain"] = np.where(df["change"]>=0, df["change"], 0) #store any gain, but store any loss as 0
    df["loss"] = np.where(df["change"]<0, -1*df["change"], 0) #store any loss *-1 (as don't want negative number), but store any gain as 0
    df["avgGain"] = df["gain"].ewm(alpha=1/n, min_periods=n).mean() #apply exponential rolliing average formula to gains and losses, specify the smoothing factor directly (alpha is 1/length(n))
    df["avgLoss"] = df["loss"].ewm(alpha=1/n, min_periods=n).mean()
    df["rs"] = df["avgGain"]/df["avgLoss"]
    df["rsi"] = 100 - (100/ (1 + df["rs"]))
    return df["rsi"]

for ticker in ohlcv_data:
    ohlcv_data[ticker]["RSI"] = RSI(ohlcv_data[ticker])

In [19]:
print(ohlcv_data["MSFT"])

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-09 09:30:00-04:00  324.750000  325.199799  323.589996  324.149994   
2023-10-09 09:35:00-04:00  324.174988  324.540009  323.179993  324.174988   
2023-10-09 09:40:00-04:00  324.183807  325.714996  323.850006  325.570007   
2023-10-09 09:45:00-04:00  325.579987  326.859985  325.470001  326.839996   
2023-10-09 09:50:00-04:00  326.829987  327.179993  326.190002  326.850006   
...                               ...         ...         ...         ...   
2023-11-08 09:30:00-05:00  361.679993  363.869995  361.404999  363.869995   
2023-11-08 09:35:00-05:00  363.850006  363.869995  362.239990  362.804993   
2023-11-08 09:40:00-05:00  362.804993  362.869995  362.040009  362.265015   
2023-11-08 09:45:00-05:00  362.339996  363.050415  362.130005  362.755005   
2023-11-08 09:50:00-05:00  362.877899  362.877899  362.877899  362.877899   

In [20]:
def ATR(DF, n=14): #default value for window of rolling mean is 14 periods
    "function to calculate True Range and Average True Range"
    df = DF.copy() #copy the df with the prices for the stock
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1)) #shift is a pandas function which allows you to compare with previous row
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False) #True Range is max of 3 columns (axis=1)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean() #Average True Range is the exponential moving average of the TR
    return df["ATR"]

for ticker in ohlcv_data:
    ohlcv_data[ticker]["ATR"] = ATR(ohlcv_data[ticker]) #apply the function to each stock

In [21]:
print(ohlcv_data["MSFT"])

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-09 09:30:00-04:00  324.750000  325.199799  323.589996  324.149994   
2023-10-09 09:35:00-04:00  324.174988  324.540009  323.179993  324.174988   
2023-10-09 09:40:00-04:00  324.183807  325.714996  323.850006  325.570007   
2023-10-09 09:45:00-04:00  325.579987  326.859985  325.470001  326.839996   
2023-10-09 09:50:00-04:00  326.829987  327.179993  326.190002  326.850006   
...                               ...         ...         ...         ...   
2023-11-08 09:30:00-05:00  361.679993  363.869995  361.404999  363.869995   
2023-11-08 09:35:00-05:00  363.850006  363.869995  362.239990  362.804993   
2023-11-08 09:40:00-05:00  362.804993  362.869995  362.040009  362.265015   
2023-11-08 09:45:00-05:00  362.339996  363.050415  362.130005  362.755005   
2023-11-08 09:50:00-05:00  362.877899  362.877899  362.877899  362.877899   

In [22]:
def ADX(DF, n=14): #default value for window of rolling mean is 14 periods
    "function to calculate ADX"
    df = DF.copy() #copy the df with the prices for the stock
    df["ATR"] = ATR(DF, n) #new column with ATR musing ethod as previously defined
    df["upmove"] = df["High"] - df["High"].shift(1) #new column which calculates current high minus previous high
    df["downmove"] = df["Low"].shift(1) - df["Low"] #new column which calculates current low minues previous low
    df["+dm"] = np.where((df["upmove"]>df["downmove"]) & (df["upmove"] >0), df["upmove"], 0) #new column with conditional value
    df["-dm"] = np.where((df["downmove"]>df["upmove"]) & (df["downmove"] >0), df["downmove"], 0) #new column with conditional value
    df["+di"] = 100 * (df["+dm"]/df["ATR"]).ewm(alpha=1/n, min_periods=n).mean() #new column with exponential moving average of +dm divided by ATR
    df["-di"] = 100 * (df["-dm"]/df["ATR"]).ewm(alpha=1/n, min_periods=n).mean() #new column with exponential moving average of -dm divided by ATR
    df["ADX"] = 100* abs((df["+di"] - df["-di"])/(df["+di"] + df["-di"])).ewm(alpha=1/n, min_periods=n).mean() #ADX is 100x the exponential moving average of the absolute value of (+di - -di)/(+di + -di)
    return df["ADX"] #return the ADX column only

for ticker in ohlcv_data:
    ohlcv_data[ticker]["ADX"] = ADX(ohlcv_data[ticker],20) #apply the function to each stock #can adjust from the default of 14 periods of required

In [ ]:
print(ohlcv_data["MSFT"])

                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-04 09:30:00-04:00  314.029999  316.600006  314.000000  316.528503   
2023-10-04 09:35:00-04:00  316.559998  317.456207  316.350006  317.130005   
2023-10-04 09:40:00-04:00  317.149811  317.269989  316.625000  316.869995   
2023-10-04 09:45:00-04:00  316.880005  317.880005  316.730011  317.364990   
2023-10-04 09:50:00-04:00  317.339996  317.525909  316.070007  316.575012   
...                               ...         ...         ...         ...   
2023-11-03 15:35:00-04:00  353.720001  353.880005  353.579987  353.839996   
2023-11-03 15:40:00-04:00  353.850006  353.869995  353.510010  353.540009   
2023-11-03 15:45:00-04:00  353.539886  353.539886  352.929993  353.029999   
2023-11-03 15:50:00-04:00  353.079987  353.269989  352.660004  352.815002   
2023-11-03 15:55:00-04:00  352.825012  353.220001  352.609985  352.809998   

In [23]:
# Import necesary libraries
import yfinance as yf
from stocktrends import Renko #use stocktrends library

In [28]:
# Download historical data for required stocks
tickers = ["MSFT","AAPL","GOOG"]
ohlcv_data = {} #empty dictionaries
hour_data = {}
renko_data = {}
# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m') #5 minute data over 1 month
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp
    temp = yf.download(ticker,period='1y',interval='1h') #hourly data over 1 year
    temp.dropna(how="any",inplace=True)
    hour_data[ticker] = temp
print(ohlcv_data["MSFT"]) #print df for MSFT

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                                 Open        High         Low       Close  \
Datetime                                                                    
2023-10-09 09:30:00-04:00  324.750000  325.199799  323.589996  324.149994   
2023-10-09 09:35:00-04:00  324.174988  324.540009  323.179993  324.174988   
2023-10-09 09:40:00-04:00  324.183807  325.714996  323.850006  325.570007   
2023-10-09 09:45:00-04:00  325.579987  326.859985  325.470001  326.839996   
2023-10-09 09:50:00-04:00  326.829987  327.179993  326.190002  326.850006   
...                               ...         .

In [29]:
def renko_DF(DF, hourly_df):
    "function to convert ohlc data into renko bricks"
    df = DF.copy() #copy the df with the prices for the stock
    #make the columns fit the required format for the renko function i.e. date, open, high, low, close
    df.reset_index(inplace=True) #date currently the index, so reset index
    df.drop("Close",axis=1,inplace=True) #drop the close price as using ajdusted close
    df.columns = ["date","open","high","low","close","volume"] #rename column headers to those required by stock trend library
    df2 = Renko(df) #create renko object of the renko class - see stocktrends library for attributes etc
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0) #3 x hourly ATR as brick size, select last value, rounded up
    renko_df = df2.get_ohlc_data() #apply the class function from the stocktrends library for each stock
    return renko_df #return the renko_df

for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker]) #apply the function to each stock

In [31]:
print(renko_data["MSFT"])

                       date   open   high    low  close  uptrend
0 2023-10-09 09:30:00-04:00  318.0  324.0  318.0  324.0     True
1 2023-10-09 13:55:00-04:00  324.0  330.0  324.0  330.0     True
2 2023-10-19 13:05:00-04:00  330.0  336.0  330.0  336.0     True
3 2023-10-25 09:30:00-04:00  336.0  342.0  336.0  342.0     True
4 2023-10-26 11:40:00-04:00  336.0  336.0  330.0  330.0    False
5 2023-11-01 09:35:00-04:00  336.0  342.0  336.0  342.0     True
6 2023-11-02 10:50:00-04:00  342.0  348.0  342.0  348.0     True
7 2023-11-03 14:10:00-04:00  348.0  354.0  348.0  354.0     True
8 2023-11-07 10:25:00-05:00  354.0  360.0  354.0  360.0     True
